In [2]:
import tkinter as tk
from tkinter import ttk
import sqlite3


class Main(tk.Frame):
    def __init__(self, root):
        super().__init__(root)
        self.init_main()
        self.db = db
        self.view_records()
    
    def init_main(self):
        toolbar = tk.Frame(bd=2)
        toolbar.pack(side=tk.TOP, fill=tk.X)
        
        btn_open_dialog = tk.Button(toolbar, text = "Добавить", command=self.open_dialog, bd = 0)
        btn_open_dialog.pack(side=tk.LEFT)
        
        
        btn_edit_dialog = tk.Button(toolbar, text='Редактировать', bd=0, command=self.open_update_dialog)
        btn_edit_dialog.pack(side=tk.LEFT)
        
        btn_delete = tk.Button(toolbar, text='Удалить', bd=0, command=self.delete_records)
        btn_delete.pack(side=tk.LEFT)
        
        self.tree = ttk.Treeview(self,columns=('ID', 'costs', 'total'), height=15, show='headings')
        
        self.tree.column('ID', width=30, anchor=tk.CENTER)
        self.tree.column('costs', width=150, anchor=tk.CENTER)
        self.tree.column('total', width=100, anchor=tk.CENTER)
        
        self.tree.heading('ID', text='ID')
        self.tree.heading('costs', text='Доход/расход')
        self.tree.heading('total', text='Сумма')
        self.tree.pack()
        
    
    def records(self, costs, total):
        self.db.insert_data(costs, total)
        self.view_records()
        
    def update_record(self, costs, total):
        self.db.c.execute('''UPDATE cash SET costs=?, total=? WHERE ID=?''',
                          (costs, total, self.tree.set(self.tree.selection()[0], '#1')))
        self.db.conn.commit()
        self.view_records()
        
    def view_records(self):
        self.db.c.execute('''SELECT * FROM cash''')
        [self.tree.delete(i) for i in self.tree.get_children()]
        [self.tree.insert('','end', values=row) for row in self.db.c.fetchall()]  
        
    def delete_records(self):
        for selection_item in self.tree.selection():
            self.db.c.execute('''DELETE FROM cash WHERE id=?''', (self.tree.set(selection_item, '#1'),))
        self.db.conn.commit()
        self.view_records()
        
        
            
    def open_dialog(self):
        Child()
        
    def open_update_dialog(self):
        Update()
    
        
        
        
class Child(tk.Toplevel):
    def __init__(self):
        super().__init__(root)
        self.init_child()
        self.view = app
        
    def init_child(self):
        self.title('Доходы/расходы')
        self.geometry('400x220+400+300')
        
        label_select = tk.Label(self, text='Доходы/расходы:')
        label_select.place(x=50, y=80)
        label_sum = tk.Label(self, text='Сумма:')
        label_sum.place(x=50, y=110)        
        
        self.entry_money = ttk.Entry(self)
        self.entry_money.place(x=200, y=110)

        self.combobox = ttk.Combobox(self, values=[u'Доход', u'Расход'])
        self.combobox.current(0)
        self.combobox.place(x=200, y=80)
        
        btn_cancel = ttk.Button(self, text='Закрыть', command=self.destroy)
        btn_cancel.place(x=300, y=170)

        self.btn_ok = ttk.Button(self, text='Добавить')
        self.btn_ok.place(x=220, y=170)
        self.btn_ok.bind('<Button-1>',lambda event: self.view.records(self.combobox.get(),
                                                                      self.entry_money.get()))      
        
        
        self.grab_set()
        self.focus_set()

class Update(Child):
    def __init__(self):
        super().__init__()
        self.init_edit()
        self.view = app

    def init_edit(self):
        self.title('Редактировать')
        btn_edit = ttk.Button(self, text='Редактировать')
        btn_edit.place(x=205, y=170)
        btn_edit.bind('<Button-1>', lambda event: self.view.update_record(self.combobox.get(),
                                                                          self.entry_money.get()))
        self.btn_ok.destroy()

        
class DB:
    def __init__(self):
        self.conn = sqlite3.connect('cash.db')
        self.c = self.conn.cursor()
        self.c.execute(
            '''CREATE TABLE IF NOT EXISTS cash (id INTEGER PRIMARY KEY, costs TEXT, total REAL)''')
        self.conn.commit()
        
    def insert_data(self, costs, total):
        self.c.execute('''INSERT INTO cash(costs, total) VALUES (?, ?)''',(costs, total))
        self.conn.commit()
    
    
if __name__ == "__main__":
    root = tk.Tk()
    db = DB()
    app = Main(root)
    app.pack()
    root.title('cash')
    root.geometry('650x450+300+200')
    root.mainloop()